# Computing the spacing of solutions comprising a Pareto frontier
Given a Pareto-optimal frontier of solutions to a multi-criterion optimization model, this program computes Schott's spacing metric (SSM). Smaller values indicate a higher density of solutions.

In [31]:
import pandas as pd
import numpy as np
import itertools

### Prepocess data and normalize the objective space

In [32]:
# Get the solutions that define the pareto frontier.
# It is assumed that these frontiers do not contain any dominated solutions
# (that is, that the solutions are truly Pareto optimal)

sols = pd.read_csv("../solutionSets/3d/ClimateChange_E85/climateChange_EfficientSolutions_E85.csv")

In [33]:
# Hard-coded manipulations to tidy-up sols

sols = sols.drop(["SolutionIndex","Frontier"],axis=1) #remove solution index and frontier columns (climate change sets)
#sols = sols.drop(["SolutionIndex"],axis=1) #remove solution index column (pack forest and Chilean sets and portfolio opt)
# nothing for some sets (toth mcDill, small sed fire)

In [34]:
# Get list of objectives from the dataframe

objs = sols.columns.tolist()

In [35]:
# Create empty dictionary to hold the objectives and the senses of each
# This process is hard-coded and requires some knowledge about the model
# that resulted in the dataframe

objSenses = {}

# Hard-coded manipulations to properly set objSenses (max/min)
# 1 for max, and 0 for min

for obj in objs:
    objSenses[obj] = 0
objSenses["Northern Spotted Owl Habitat (ha)"] = 1
#objSenses["PER"] = 0
#objSenses

In [36]:
def maxAll(origf,objSenses):
    
    f = origf
    
    # get bounds on the objectives' values
    objBounds = {}
    for obj,sense in objSenses.items():
        objBounds[obj] = [f[obj].min(),f[obj].max()]
    # get each objective's ideal value
    idealObjVals = {}
    for obj,bounds in objBounds.items():
        idealObjVals[obj] = bounds[objSenses[obj]] # 0th entry of bounds is min, 1st is max
    # get each objective's worst value
    worstObjVals = {}
    for obj,bounds in objBounds.items():
        worstObjVals[obj] = bounds[not(objSenses[obj])]
        
    # Convert each objective to a maximization

    for obj,bounds in objBounds.items():
        f[obj] = f[obj].apply(
            lambda x: \
            abs(x - worstObjVals[obj]))
    
    return f

In [37]:
def maxAllAndNorm(origf,objSenses):
    
    f = origf
    
    # get bounds on the objectives' values
    objBounds = {}
    for obj,sense in objSenses.items():
        objBounds[obj] = [f[obj].min(),f[obj].max()]
    # get each objective's ideal value
    idealObjVals = {}
    for obj,bounds in objBounds.items():
        idealObjVals[obj] = bounds[objSenses[obj]] # 0th entry of bounds is min, 1st is max
    # get each objective's worst value
    worstObjVals = {}
    for obj,bounds in objBounds.items():
        worstObjVals[obj] = bounds[not(objSenses[obj])]
        
    # Normalize the objective spaces by converting each value to the relative achievement along its axis:

    #   distance from the worst case value
    # ---------------------------------------
    # total distance spanned by the objective

    # Normalized objective space is in [0,1]^N where N = # of objectives


    for obj,bounds in objBounds.items():
        f[obj] = f[obj].apply(
            lambda x: \
            abs(x - worstObjVals[obj]) / \
            (bounds[1]-bounds[0]))
    
    return f

In [38]:
# Convert/normalize the data

normalize = False

if normalize:
    sols = maxAllAndNorm(sols,objSenses)
else:
    sols = maxAll(sols,objSenses)

### Computing the spacing of solutions along the frontier

In [39]:
# Compute the d_i's and average dbar


sols["di"] = np.infty
for idx,row in sols.iterrows():
    minDist = sols.ix[idx]["di"]
    for idx2,row2 in sols.iterrows():
        if idx2 == idx:
            continue
        dist = np.linalg.norm(row[objs]-row2[objs])
        if dist < minDist:
            minDist = dist
    sols.set_value(idx,"di",minDist)
dbar = np.mean(sols["di"])

In [40]:
# Compute the spacing metric


spacing = np.sqrt(sum(np.power(sols["di"]-dbar,2))/(len(sols)-1))
spacing

6.3138151010280978